In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import matplotlib.pyplot as plt

from src.destot.DESTOT import align, xi_to_growth_rate
from src.destot.metrics import growth_distortion_metric, migration_metric

def plot_slice_value(slice, value_vec, vmax=None, vmin=None):
    """
    Parameters: 
    slice: AnnData object of the slice
    value_vec: growth vector

    Returns:
    Plots the slice with each spot colored according to its value in value_vec
    """
    plt.figure()
    spatial = slice.obsm['spatial']
    sc = plt.scatter(spatial[:, 0], spatial[:, 1], c=value_vec, cmap='RdYlGn', s=50, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(sc)
    cbar.ax.tick_params(labelsize=20)
    plt.gca().invert_yaxis()
    plt.axis('off')

    fig = plt.gcf()
    fig_size = fig.get_size_inches()
    new_width = 20.0
    new_height = new_width * (fig_size[1] / fig_size[0])
    fig.set_size_inches(new_width, new_height)
    plt.show()
    return

# Loading the Axolotl Telencephalon Slice Pair

The source for the stage 54 and stage 57 embryonic development datasets is https://db.cngb.org/stomics/artista/download/ under the file names Stage54.h5ad and Stage57.h5ad, which can be downloaded directly from the portal.

Alternatively, you can specify your directory of choice in the terminal and run the commands:

`wget https://ftp.cngb.org/pub/SciRAID/stomics/STDS0000056/stomics/Stage54.h5ad target_directory`

`wget https://ftp.cngb.org/pub/SciRAID/stomics/STDS0000056/stomics/Stage57.h5ad target_directory`

In [3]:
target_directory = '/home/ph3641/TDA/axolotl/'

slice_54 = sc.read_h5ad(target_directory+'Stage54.h5ad')
slice_57 = sc.read_h5ad(target_directory+'Stage57.h5ad')

Our method assumes the two input AnnData objects have its gene expression count matrix stored in the `.X` field, while in this dataset they are stored in `.layers['counts']`. If one's AnnData object assumes a different key, one can easily change the reference to move the count matrix to `.X`.

In [4]:
slice_54.X = slice_54.layers['counts']
slice_57.X = slice_57.layers['counts']

The function `align` is the main function for running DeST-OT on a pair of AnnData objects. If one has the cost matrices precomputed in pytorch, one can directly call `destot_opt.LogSinkhorn_iteration` to return an alignment.

In [ ]:
Pi, xi = align(slice_54, slice_57, alpha=0.2, gamma=50, epsilon=0.1, max_iter=100, 
               balanced=False, use_gpu=False, normalize_xi=True, check_convergence=False)

/home/ph3641/.conda/envs/peterenv2/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/ph3641/.conda/envs/peterenv2/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/ph3641/.conda/envs/peterenv2/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
/home/ph3641/.conda/envs/peterenv2/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call 

Iteration: 0
Iteration: 5
Iteration: 10
Iteration: 15
Iteration: 20
Iteration: 25
Iteration: 30
Iteration: 35
Iteration: 40
Iteration: 45


In [ ]:
plot_slice_value(slice_54, xi)

# Metrics:

DeST-OT quantifies two metrics for assessing the biological plausibility of a _spatiotemporal_ alignment, the _growth-distortion metric_ and the _migration metric_. 

- The growth-distortion metric, denoted $\mathcal{J}_{\textrm{growth}}(\xi)$, quantifies the accuracy of the growth-rates learned from the alignment $\Pi$ using a ground-truth partition of the cell-types returned from any clustering algorithm.


- The migration metric, denoted $\mathcal{J}_{\textrm{migration}}(\Pi)$, quantifies the amount of spatial migration (distance travelled) by ancestor cells to their descendants under an alignment

One can easily compute the migration metric or growth-distortion metric for any alignment $\Pi$. This alignment can come from any spatiotemporal alignment method.

In [ ]:
print(f'The cell migration distance between stage 54 and 57 is: {migration_metric(slice_54, slice_57, Pi)}')

There are two options for computing the growth rate distortion.

- A growth-rate distortion assuming diagonal transitions (i.e. that cells do not transition between types, as might be the case in non-developmental processes). This can be set with the argument `option="no_transition"`.


- The growth-rate distortion which the lowest possible under _any_ cell type transition. This option depends on the transition matrix which most accurately reflects the observed growth rates. This can be set with the argument `option="infer_transition"` $-$ we use this as a default and suggest this setting.

In [ ]:
print(f'The growth-rate distortion of the DeST-OT alignment is: {growth_distortion_metric(slice_54, slice_57, Pi, annotation_key="Annotation")}')

In [ ]:
import numpy as np

plt.rcParams["figure.figsize"] = [6.50, 6.50]
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 300

print(xi)

N1 = xi.shape[0]
Js = np.log(N1*xi + 1)
print(np.min(Js))

In [ ]:
Js = xi_to_growth_rate(xi, t1=54, t2=57, normalize_xi=True)

plt.hist(Js, bins=100)
plt.axvline(x=0, c='r', label='Line Separating Growth ($J_{i} > 0$) from Death ($J_{i} < 0$)')
plt.xlabel('Growth Rates $J_{i}$ in units of $\mathrm{Stage}^{-1}$')
plt.legend()
plt.show()

plt.hist(xi, bins=100)
plt.axvline(x=0, c='r', label=r'Line Separating Growth ($\mathbf{\xi}_{i} > 0$) from Death ($\mathbf{\xi}_{i} < 0$)')
plt.xlabel(r'Growth $\mathbf{\xi}_{i}$ in units of Descendant Cells between the two time-points')
plt.legend()
plt.show()